# SkymmapDemo
Demonstration of some skymap operations with GBMCMC pandas data set


In [ ]:
# Import modules
import os
import pandas as pd
import numpy as np  
import healpy as hp
from astropy.coordinates import SkyCoord
from astropy import units as u
import ligo.skymap.plot
import matplotlib.pyplot as plt
import lisacattools as lisacat
%matplotlib inline

In [ ]:
# Start by loading the main catalog file processed from GBMCMC outputs
catFile = '/home/centos/Data/cat15728640_v2.h5'
catPath = os.path.split(catFile)[0]
cat = pd.read_hdf(catFile, key='detections')
cat.head()

## Build the all-sky map
Run through all the sources and make an all-sky map of all the chain samples 
can take a while to run, you can save the output to speed things up later using the *recomplie* flag

In [ ]:

recompile = False
allSkyFileName = '/home/centos/Data/cat15728640_v2_allSky.h5'
if recompile :
    cat.sort_values(by='SNR',ascending = False, inplace=True)
    sources = list(cat.index)
    dfs = list()
    for source in sources:
        df = lisacat.getChain(cat,source,catPath)
        lisacat.getGalcoord(df)
        df.insert(len(df.columns),'Source',source,True)
        df.insert(len(df.columns),'Chain Length',len(df),True)
        dfs.append(df[['Source','Frequency','Frequency Derivative','Amplitude','Galactic Latitude','Galactic Longitude','Chain Length']])

    allSources = pd.concat(dfs)
    allSources.to_hdf(allSkyFileName,key='sources')
else :
    allSources = pd.read_hdf(allSkyFileName,key='sources')


allSources.describe()

In [ ]:
# bin all the sources in a healpix bin 
nside = 64
lisacat.HPbin(allSources,nside,'Galactic')
allSources

## Find the Targets
Next we find source(s) in a given circle. This will return a dataframe containing a list of sources who have chain samples inside the circle. 
The list will be ranked by *Probability*, which is the fraction of chain samples that are inside the circle for that particular source.  
The parameters for the sources are the median parameters from the samples *inside* the circle (e.g. they are restricted to that sky locaiton).
The dataframe selBins is a subset of the full data set corresponding to only that part of the sky, which could be useful for other processing.

This may also take a while!

In [ ]:
ra = -95  # right ascension of the circle's center (degrees)
dec = -10  # declination of the circle's center (degrees)
radius = 5  # radius of the circle (degrees)
theta = 0.5*np.pi-np.deg2rad(dec)
phi = np.deg2rad(ra)
radius = np.deg2rad(radius)
xyz = hp.ang2vec(theta,phi)
ipix_disc = hp.query_disc(nside,xyz,radius)
selBins = allSources[allSources['HEALPix bin'].isin(ipix_disc)]
cnts = selBins['Source'].value_counts()
df_cnts = pd.DataFrame(cnts)
df_cnts.rename(columns={'Source':'Counts'},inplace=True)
cutidx = df_cnts[df_cnts['Counts']<100].index
df_cnts.drop(index=cutidx,inplace=True)
idxs = list(df_cnts.index)
srcs = list()
for idx in idxs:
    meds = selBins[selBins['Source']==idx].median()
    meds['Source'] = idx
    srcs.append(meds)

df_targets = pd.DataFrame(srcs)
df_targets.set_index('Source',inplace=True)
df_targets = pd.concat([df_targets,df_cnts],1)
df_targets['Probability'] = df_targets['Counts']/df_targets['Chain Length']
df_targets.drop(columns=['Counts','Chain Length','HEALPix bin'],inplace=True)
df_targets = df_targets[['Probability', 'Galactic Latitude', 'Galactic Longitude', 'Amplitude','Frequency','Frequency Derivative']]
df_targets.sort_values(by = 'Probability',ascending = False, inplace=True)
df_targets

## Make some Cool Skymaps
Use the ligo.skymap package to make a neat plot

In [ ]:
# all sky map
hpmap = lisacat.HPhist(allSources,nside)
fig = plt.figure(figsize=(8, 6), dpi=100)

ax = plt.axes(
    [0.05, 0.05, 0.9, 0.9],
    projection='geo degrees mollweide')
ax.grid()
ax.imshow_hpx(np.log10(hpmap+1), cmap='plasma')


In [ ]:
# A zoom-in map showing the target circle
fig = plt.figure(figsize=(6, 6), dpi=100)
center = SkyCoord(ra, dec, unit = u.deg)
ax = plt.axes(
    [0.05, 0.05, 0.9, 0.9],
    projection='astro globe',
    center=center)

ax_inset = plt.axes(
    [0.59, 0.3, 0.4, 0.4],
    projection='astro zoom',
    center=center,
    radius=10*u.deg)

for key in ['ra', 'dec']:
    ax_inset.coords[key].set_ticklabel_visible(False)
    ax_inset.coords[key].set_ticks_visible(False)
ax.grid()
ax.mark_inset_axes(ax_inset)
ax.connect_inset_axes(ax_inset, 'upper left')
ax.connect_inset_axes(ax_inset, 'lower left')
ax_inset.scalebar((0.1, 0.1), 5 * u.deg).label()
ax_inset.compass(0.9, 0.1, 0.2)

ax.imshow_hpx(np.log10(hpmap+1), cmap='plasma')
ax_inset.imshow_hpx(np.log10(hpmap+1), cmap='plasma')
ax_inset.add_artist(plt.Circle((center.ra.deg,center.dec.deg),np.rad2deg(radius), fill = False, transform=ax_inset.get_transform('world')))